In [76]:
import requests
import xml.etree.ElementTree as ET 
import xml.sax
import pandas as pd

In [77]:
Keys = [0] * 13
country = 'USA'
year = 2024
month = 12
term_length = 4

**Keys 1: Party Mandate**

Key 1 is turned True if the incumbent Party has had a net gain of seats from the previous Presidential and Midterm elections combined. In other words, does the incumbent party hold more seats than they held after the midterm elections of the previous presidency?

In 2024, the Democrats held 213 seats versus 235 in in 2018. Therefore, the key is turned False.

In [78]:
Keys[0] = 0

**Keys 2, 3, 4: Primary Contest, Incumbent President, Third Party**

Keys 2, 3, and 4 are difficult to quantify, and while be up to the user to determine. However, these should be quite objective, since keys 2, 3, and 4 are:

- No Primary Contest

- Incumbent Seeking Re-Election

- No Third Party

In the 2024 Election, these were the respective results for each of the given keys.

No Primary Contest - True. Due to Joe Biden dropping out during the summer of '24, the Democratic Party did not have the time to run a proper primary. Therefore, there was no primary contest. Hence, Key 2 is turned True.

Incumbent Seeking Re-Election - False. Although Harris was the VP, she was not a president seeking re-election. Therefore, she was NOT an incumbent seeking re-election.

No Third Party - In Lichtman's model, this key is turned true if there is a third party candidate polling consistently above 10% in the polls. High support for a third party indicates dissatisfaction with the incumbent party. In 2024, this was True.

In [79]:
Keys[1] = 1
Keys[2] = 0
Keys[3] = 1

**Keys 5: Short Term Economy**

Is the country in a recession during the election campaign? 

A Recession is two consecutive quarters of GDP decline. 

-> Check average GDP of three quarters, check if decreasing -> Recession

In [80]:
GDP = pd.read_excel(country + ".xlsx", sheet_name= 'Quarterly')

GDP.columns = ['Date', 'GDP']

GDP.index = GDP.Date

GDP.drop(columns = 'Date', inplace=True)

display(GDP)

,GDP
Date,
1947-01-01,2169.189
1947-04-01,2167.121
1947-07-01,2169.565
1947-10-01,2190.947
1948-01-01,2250.560
...,...
2023-07-01,22602.433
2023-10-01,22880.288
2024-01-01,23035.857


In [81]:
months = pd.DataFrame()

quarters_taken = 8

take_month = month - 9

if take_month == 1:
    starting_month = '-12'
    start_year = year - 2

elif take_month - 1 < 10:
    starting_month = '-0' + str(take_month - 1)
    start_year = year -1
else:
    starting_month = '-' + str(take_month - 1)
    start_year = year - 1


months['GDP'] = GDP[(GDP.index >= str(start_year) + starting_month) & (GDP.index <= str(year+1))]

months['GDP Growth'] = months.GDP.pct_change().ffill()
months.drop(index=months.index[0], inplace=True)

display(months)

,GDP,GDP Growth
Date,,
2023-07-01,22602.433,0.006466
2023-10-01,22880.288,0.012293
2024-01-01,23035.857,0.006799
2024-04-01,23132.662,0.004202
2024-07-01,23217.692,0.003676


In [82]:
decreased_month = 0
recession = False

for i in range(1, len(months.index)):
    ind = months.index[i]

    if months.loc[ind, 'GDP Growth'] < 0:
        decreased_month += 1

        if decreased_month >= 2:
            recession = True
            i = len(months.index)

    else:
        decreased_month = 0

if recession:
    print('Key 5 is False. There was a recession during the election campaign.')
    

else:
    print('Key 5 is True. There was no recession during the election campaign.')
    Keys[4] = 1

Key 5 is True. There was no recession during the election campaign.


**Key 6: Long Term Economy**

Real per Capita economic growth during term >= mean growth during previous 2 terms means that strong economic growth

In [83]:


#last two terms
start = year - 13

GDP_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/NY.GDP.PCAP.CD" + '?date=' + str(start) + ':' + str(year)

inflation_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/NY.GDP.DEFL.KD.ZG" + '?date=' + str(start) + ':' + str(year)

gdp_file = requests.get(GDP_api).text

inflation_file = requests.get(inflation_api).text

print(inflation_file)

ï»¿<?xml version="1.0" encoding="utf-8"?>
<wb:data page="1" pages="1" per_page="50" total="13" sourceid="2" lastupdated="2024-12-16" xmlns:wb="http://www.worldbank.org">
  <wb:data>
    <wb:indicator id="NY.GDP.DEFL.KD.ZG">Inflation, GDP deflator (annual %)</wb:indicator>
    <wb:country id="US">United States</wb:country>
    <wb:countryiso3code>USA</wb:countryiso3code>
    <wb:date>2023</wb:date>
    <wb:value>3.59839135378053</wb:value>
    <wb:unit />
    <wb:obs_status />
    <wb:decimal>1</wb:decimal>
  </wb:data>
  <wb:data>
    <wb:indicator id="NY.GDP.DEFL.KD.ZG">Inflation, GDP deflator (annual %)</wb:indicator>
    <wb:country id="US">United States</wb:country>
    <wb:countryiso3code>USA</wb:countryiso3code>
    <wb:date>2022</wb:date>
    <wb:value>7.12948085421044</wb:value>
    <wb:unit />
    <wb:obs_status />
    <wb:decimal>1</wb:decimal>
  </wb:data>
  <wb:data>
    <wb:indicator id="NY.GDP.DEFL.KD.ZG">Inflation, GDP deflator (annual %)</wb:indicator>
    <wb:country i

In [84]:
def calculate_inflation(rates):
    inflation = pd.DataFrame(columns = ['inflation'], index = rates.index)
    
    for i in inflation.index:    
        inflation.loc[i, 'inflation'] = 1
        
        for j in range(int(rates.index[0]), int(i)):
            inflation.loc[i, 'inflation']  += rates[str(j)]

            #print(rates[str(j)])

    return inflation

In [85]:
country_data = pd.DataFrame(columns = ['Date', 'Value', 'inflation', 'Real Value', 'Real Increase'])



class XMLHandler(xml.sax.ContentHandler):
    
    def __init__(self):
        self.CurrentData = ""
        self.country = ""
        self.date = ""
        self.value = ""
        self.count = 0

    def startElement(self, tag, attribute):
        self.CurrentData = tag

        if tag == "wb:data":
            self.count += 1

    def endElement(self, tag):
        
        if self.CurrentData == "wb:date":

            country_data.loc[self.count, 'Date'] = self.date
        
        elif self.CurrentData == "wb:value":
            
            country_data.loc[self.count, 'Value'] = float(self.value)

    def characters(self, content):
        if self.CurrentData == "wb:date":
            self.date = content
        elif self.CurrentData == "wb:value":
            self.value = content

parser = xml.sax.make_parser()

parser.setFeature(xml.sax.handler.feature_namespaces, 0)

Handler = XMLHandler()

parser.setContentHandler( Handler )
parser.parse(GDP_api)

country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

gdp_data = country_data.copy()

country_data = pd.DataFrame(columns = ['Date', 'Value'])

parser.parse(inflation_api)

country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

inflation_data = country_data/100

gdp_data.inflation = calculate_inflation(inflation_data.Value)

display(gdp_data)

,Value,inflation,Real Value,Real Increase
Date,,,,
2011,50065.978947,1,NaN,NaN
2012,51784.411469,1.020631,NaN,NaN
2013,53409.750777,1.039253,NaN,NaN
2014,55304.315532,1.056272,NaN,NaN
2015,57040.208214,1.073688,NaN,NaN
2016,58206.614193,1.082968,NaN,NaN
2017,60322.261424,1.092471,NaN,NaN
2018,63201.045848,1.110379,NaN,NaN
2019,65604.68152,1.133288,NaN,NaN


In [86]:
gdp_data['Real Value'] = gdp_data.Value/gdp_data.inflation

gdp_data['Real Increase'] = gdp_data['Real Value'].pct_change().ffill()

gdp_data.drop(index=gdp_data.index[0], inplace=True)

display(gdp_data)

C:\Users\paliu\AppData\Local\Temp\ipykernel_29536\1300146166.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdp_data['Real Increase'] = gdp_data['Real Value'].pct_change().ffill()


,Value,inflation,Real Value,Real Increase
Date,,,,
2012,51784.411469,1.020631,50737.638443,0.013415
2013,53409.750777,1.039253,51392.455862,0.012906
2014,55304.315532,1.056272,52358.047185,0.018789
2015,57040.208214,1.073688,53125.510069,0.014658
2016,58206.614193,1.082968,53747.29983,0.011704
2017,60322.261424,1.092471,55216.331044,0.027332
2018,63201.045848,1.110379,56918.423618,0.030826
2019,65604.68152,1.133288,57888.783078,0.017048
2020,64411.373178,1.149793,56019.97462,-0.032283


In [87]:
#Now calculate the average increase for each term

year = int(gdp_data.index[0])

avg_increases = [0] * 3

for i in range(len(avg_increases)):
    for j in range(term_length):
        avg_increases[i] += gdp_data.loc[str(year), "Real Increase"]
        year += 1

    avg_increases[i] /= term_length

#print(avg_increases)

pre_term_increase = avg_increases[0] + avg_increases[1] 
pre_term_increase /= 2

if pre_term_increase < avg_increases[-1]:
    print("Key 6 is True. The incumbent party has had a strong long term economy")
    Keys[5] = 1

else:
    print("Key 6 is False. The incumbent party has had a weak long term economy")
    Keys[5] = 0

Key 6 is True. The incumbent party has had a strong long term economy


**Key 7: Policy Change**

To determine this, we will analyze the number of legislative bills passed through Congress throughout the two half-terms of the current presidency. If this is less than 1 standard deviation of the mean number of bills passed, then the term has not brought about a major policy change.

Data was drawn from: https://www.govtrack.us/congress/bills/statistics 

In [88]:
Bills = pd.read_excel(country + "-bills.xlsx", sheet_name= 'Quarterly')

Bills.columns = ['Date', 'End', 'Bills']

Bills.index = Bills.Date

Bills.drop(columns = 'Date', inplace=True)


#Compare to the previous 10 presidencies

start = year - 10 * 4

values_used = Bills[(Bills.index > start) & (Bills.index <= year)]

display(values_used)

,End,Bills
Date,,
1985,1986,687
1987,1988,761
1989,1990,665
1991,1992,610
1993,1994,473
1995,1996,337
1997,1998,404
1999,2000,604
2001,2002,383


In [89]:
bills_by_presidency = pd.DataFrame(columns = ['Bills'])

for i in range(len(values_used.index)):
    if i % 2 == 0:
        bills_by_presidency.loc[values_used.index[i], 'Bills'] = values_used.loc[values_used.index[i], 'Bills']

    else:
        bills_by_presidency.loc[values_used.index[i-1], 'Bills'] += values_used.loc[values_used.index[i], 'Bills']

display(bills_by_presidency)

,Bills
1985,1448
1989,1275
1993,810
1997,1008
2001,887
2005,943
2009,669
2013,625
2017,787
2021,523


In [90]:
if bills_by_presidency.Bills[bills_by_presidency.index[-1]] < bills_by_presidency.Bills.mean() - bills_by_presidency.Bills.std():
    print("There was no major policy change.")
    print("Key 7 is False.")
    Keys[6] = 0

else:
    print("There was a major policy change.")
    print("Key 7 is True.")
    Keys[6] = 1

There was no major policy change.
Key 7 is False.


**Key 8: Social Unrest**

Has there been social unrest during the previous term? If it is low, then the key is True, otherwise the key is False and there is risk of the incumbent party losing power.

We will use the World Bank's measure for Political Stability and Absence of Violence/Terrorism. If there is any term where the measure was 1 std below the mean measure, then that term is deemed to have had social unrest and therefore, the key is turned False.

In [91]:
start = year -10

unrest_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/PV.EST" + '?date=' + str(start) + ':' + str(year)

In [92]:


country_data = pd.DataFrame(columns = ['Date', 'Value'])

parser.parse(unrest_api)
country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

unrest_data = country_data

display(unrest_data)

,Value
Date,
2014,0.58242
2015,0.66289
2016,0.385635
2017,0.262138
2018,0.386239
2019,0.118299
2020,-0.0279
2021,-0.014425
2022,0.008701


In [93]:
#Take the average of the last 10 years
unrest_avg = unrest_data.Value.mean()
unrest_std = unrest_data.Value.std()

print(unrest_avg)
print(unrest_std)

unrest = False

display(unrest_data)

for i in range(year - term_length + 1, year):
    
    if unrest_data.Value[str(i)] < unrest_avg - unrest_std:
        unrest = True
        print("There was unrest in", i)
        print("Key 8 is False.")
        Keys[7] = 0

if not unrest:
    print("There was no unrest.")
    print("Key 8 is True.")
    Keys[7] = 1

0.23934221602976322
0.2556741267497329


,Value
Date,
2014,0.58242
2015,0.66289
2016,0.385635
2017,0.262138
2018,0.386239
2019,0.118299
2020,-0.0279
2021,-0.014425
2022,0.008701


There was no unrest.
Key 8 is True.


**Key 9: Scandal**

Has there been a major scandal during the previous term? If it is low, then the key is True, otherwise the key is False and there is risk of the incumbent party losing power.

We will use the World Bank's measure fosm. If there is any term where the measure was 1 std below the mean measure, then that term is deemed to have had social unrest and therefore, the key is turned False.

In [94]:
start = year -10

unrest_api = "https://api.worldbank.org/v2/country/" + country + "/indicator/CC.EST" + '?date=' + str(start) + ':' + str(year)

In [95]:
country_data = pd.DataFrame(columns = ['Date', 'Value'])

parser.parse(unrest_api)
country_data.index = country_data.Date

country_data.drop(columns = 'Date', inplace = True)

country_data.sort_values(by='Date', ascending=True, inplace=True)

unrest_data = country_data

display(unrest_data)

,Value
Date,
2014,1.371378
2015,1.357721
2016,1.330896
2017,1.340533
2018,1.292747
2019,1.180357
2020,1.037832
2021,1.019278
2022,1.104145


In [96]:
#Take the average of the last 10 years
unrest_avg = unrest_data.Value.mean()
unrest_std = unrest_data.Value.std()

print(unrest_avg)
print(unrest_std)

unrest = False

display(unrest_data)

for i in range(year - term_length + 1, year):
    
    if unrest_data.Value[str(i)] < unrest_avg - unrest_std:
        unrest = True
        print("There was high corruption in", i)
        print("Key 9 is False.")
        Keys[8] = 0

if not unrest:
    print("There was low corruption.")
    print("Key 9 is True.")
    Keys[8] = 1

1.2158318758010869
0.13809047598090146


,Value
Date,
2014,1.371378
2015,1.357721
2016,1.330896
2017,1.340533
2018,1.292747
2019,1.180357
2020,1.037832
2021,1.019278
2022,1.104145


There was high corruption in 2021
Key 9 is False.


**Keys 10 and 11**

Keys 10 and 11 deals with foreign and military matters. Success in diplomacy is also difficult to quantify, so these two keys are also subjective. 

With regards to the 2024 election, Biden's presidency did not achieve a major foreign success, Americans remain divided on the results of the major foreign crisises in the US - whether or not Afghanistan, Israel, or Ukraine were viewed as a success varies across the US. I believe that for something to be viewed as a success or failure, at least 50% of the results must say that the situation was handled excellently or good to be qualified as a success, while at least 50% must say the situation was handled poorly to be qualified as a foreign failure.

Therefore, Key 10 - No Foreign Military Failure is True, and Key 11 - Foreign Military Success, is False. 

Sources:

- https://www.pewresearch.org/global/2024/05/08/views-of-ukraine-and-u-s-involvement-with-the-russia-ukraine-war/
 
- https://www.pewresearch.org/short-reads/2022/08/17/a-year-later-a-look-back-at-public-opinion-about-the-u-s-military-exit-from-afghanistan/

- https://www.pewresearch.org/short-reads/2024/10/01/slight-uptick-in-americans-wanting-u-s-to-help-diplomatically-resolve-israel-hamas-war/

In [97]:
Keys[10] = 1
Keys[11] = 0

**Keys 12 and 13**

These last two keys, candidate charisma, are by far the most subjective. 

Key 12: Is the incumbent party candidate charismatic or a national hero? I believe that Harris was NOT charismatic, so this key is turned False.

Key 13: Is the challenging party candidate charismatic or a national hero? I believe that Trump could be considered charismatic, so this key is turned True.

In [98]:
Keys[11] = 0
Keys[12] = 0

In [99]:
print(Keys)

[0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0]


In [100]:
keys_true = 0

for i in Keys:
    keys_true += i

if keys_true > 8:
    print("The incumbent party will win")
else:
    print("The challenging party will win")

The challenging party will win
